In [ ]:
# --- COLAB SETUP START ---
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # 1. Clone the repository
    !git clone https://github.com/google-gemini/gemini-fullstack-langgraph-quickstart
    %cd gemini-fullstack-langgraph-quickstart/backend

    # 2. Prepare Environment (Resolving Conflicts)
    import sys
    print("Uninstalling conflicting pre-installed packages...")
    !pip uninstall -y google-ai-generativelanguage tensorflow grpcio-status

    # Pre-install PyTorch Nightly if Python 3.13+ is detected
    if sys.version_info >= (3, 13):
        print("Detected Python 3.13+. Installing PyTorch Nightly...")
        !pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu

    # 3. Install Backend
    !pip install .

    # 4. Set API Key
    import os
    from google.colab import userdata
    try:
        os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
    except:
        print("Please enter your Gemini API Key:")
        os.environ["GEMINI_API_KEY"] = input()
# --- COLAB SETUP END ---

In [ ]:
# --- MODEL CONFIGURATION ---
# @title Select Gemini Model
# @markdown Choose the model strategy to use.
# @markdown *Note: Experimental models (2.0/2.5) may require billing/quota enablement.*

MODEL_STRATEGY = "Free Tier (Gemini 1.5 Flash)" # @param ["Free Tier (Gemini 1.5 Flash)", "Experimental (Gemini 2.0 Flash)", "Flash Lite (Gemini 2.0 Flash-Lite)", "Gemini 2.5 Flash (Experimental)", "Gemini 2.5 Pro (Experimental)"]

import os

# Map selection to actual model names
if MODEL_STRATEGY == "Free Tier (Gemini 1.5 Flash)":
    SELECTED_MODEL = "gemini-1.5-flash"
    print("✅ Using Free Tier Model: gemini-1.5-flash")
elif MODEL_STRATEGY == "Experimental (Gemini 2.0 Flash)":
    SELECTED_MODEL = "gemini-2.0-flash-exp"
    print("⚠️ Using Experimental Model: gemini-2.0-flash-exp (Quota Required)")
elif MODEL_STRATEGY == "Flash Lite (Gemini 2.0 Flash-Lite)":
    SELECTED_MODEL = "gemini-2.0-flash-lite-preview-02-05"
    print("⚠️ Using Preview Model: gemini-2.0-flash-lite-preview-02-05 (Quota Required)")
elif MODEL_STRATEGY == "Gemini 2.5 Flash (Experimental)":
    SELECTED_MODEL = "gemini-2.5-flash"
    print("🔥 Using Gemini 2.5 Flash: gemini-2.5-flash (Quota Required)")
elif MODEL_STRATEGY == "Gemini 2.5 Pro (Experimental)":
    SELECTED_MODEL = "gemini-2.5-pro"
    print("🔥 Using Gemini 2.5 Pro: gemini-2.5-pro (Quota Required)")

# Set Environment Variables to override defaults
os.environ["QUERY_GENERATOR_MODEL"] = SELECTED_MODEL
os.environ["REFLECTION_MODEL"] = SELECTED_MODEL
os.environ["ANSWER_MODEL"] = SELECTED_MODEL
os.environ["TOOLS_MODEL"] = SELECTED_MODEL

# Deep Research Agent Demo

This notebook demonstrates the Deep Research capabilities, including the planning and research loop.

In [ ]:
# Setup environment
import os
import sys
from dotenv import load_dotenv

sys.path.append(os.path.abspath("../backend/src"))
load_dotenv("../backend/.env")

## 1. Research Planning Mode

Demonstrating how the agent creates a plan before executing research.

In [ ]:
from agent.graph import graph
from langchain_core.messages import HumanMessage

async def run_planning_demo():
    # Enable planning mode via configuration
    # Note: Models are determined by environment variables we set in the config cell
    config = {
        "configurable": {
            "thread_id": "demo_planning_1",
            "require_planning_confirmation": True
        }
    }
    
    input_text = "Research the latest advancements in solid state batteries from 2024."
    print(f"Request: {input_text}\n")
    
    # Send initial request to trigger planning
    async for event in graph.astream_events(
        {"messages": [HumanMessage(content=input_text)]},
        config,
        version="v2"
    ):
        if event["event"] == "on_chat_model_stream":
            # Print streaming output (likely the plan)
            print(event["data"]["chunk"].content, end="", flush=True)

await run_planning_demo()